Import

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

In [2]:
train_data = pd.read_csv("train.csv")

## Data preproccessing

In [3]:
def data_preproccessing(df):
    for i in df.columns:
        if df[i].dtypes == 'object':
            df[i].fillna(df[i].mode()[0], inplace=True)
        else:
            df[i].fillna(df[i].median(), inplace=True)
    def ben(x):
        if x == "True":
            return(1)
        else:
            return(0)
    df['VIP'] = df['VIP'].map(lambda x: ben(x))
    df['CryoSleep'] = df['CryoSleep'].map(lambda x: ben(x))
    
    le=LabelEncoder()
    #f['Transported']= le.fit_transform(df['Transported'])
    df['HomePlanet'] = le.fit_transform(df['HomePlanet'])
    df['Cabin'] = le.fit_transform(df['Cabin'])
    df['Destination'] =le.fit_transform(df['Destination'])
    
    df.drop(["Name"], axis = 1, inplace = True)
    return(df)

In [6]:
for i in train_data.columns:
    if train_data[i].dtypes == 'object':
        train_data[i].fillna(train_data[i].mode()[0], inplace=True)
    else:
        train_data[i].fillna(train_data[i].median(), inplace=True)

In [9]:
def ben(x):
    if x == "True":
        return(1)
    else:
        return(0) 

train_data['VIP'] = train_data['VIP'].map(lambda x: ben(x))
train_data['CryoSleep'] = train_data['CryoSleep'].map(lambda x: ben(x))

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   int64  
 3   Cabin         8693 non-null   object 
 4   Destination   8693 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8693 non-null   int64  
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
 12  Name          8693 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), int64(2), object(5)
memory usage: 891.5+ KB


In [15]:
state = 12  
test_size = 0.30  
  


X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=test_size, random_state=state)

## Training GBMClassifier

In [16]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

In [17]:
for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=250, learning_rate=learning_rate, max_features=2, max_depth=5, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

Learning rate:  0.05
Accuracy score (training): 0.857
Accuracy score (validation): 0.783
Learning rate:  0.075
Accuracy score (training): 0.880
Accuracy score (validation): 0.791
Learning rate:  0.1
Accuracy score (training): 0.901
Accuracy score (validation): 0.781
Learning rate:  0.25
Accuracy score (training): 0.963
Accuracy score (validation): 0.781
Learning rate:  0.5
Accuracy score (training): 0.985
Accuracy score (validation): 0.764
Learning rate:  0.75
Accuracy score (training): 0.993
Accuracy score (validation): 0.770
Learning rate:  1
Accuracy score (training): 0.995
Accuracy score (validation): 0.748


In [18]:
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.25, max_features=2, max_depth=10, random_state=0)
gb_clf.fit(X_train, y_train)

print("Learning rate: ", learning_rate)
print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_val, y_val)))

Learning rate:  1
Accuracy score (training): 0.997
Accuracy score (validation): 0.780


In [19]:
import xgboost as xgb

# Training xgboost

In [20]:
X = X_train.drop("PassengerId",axis = 1)

In [21]:
X_v = X_val.drop("PassengerId",axis = 1)

In [22]:
results_dict = {}

In [23]:
xgbc0 = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='hist',
                          use_label_encoder=False)

In [24]:
default_params = {}
gparams = xgbc0.get_params()


In [25]:
xgbc1 = xgbc0

In [26]:

xgbc0.fit(X, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='auc', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='hist', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [27]:
score = xgbc0.score(X_v, y_val)
print(score)

0.781058282208589


In [28]:
clf0 = GridSearchCV(estimator=xgbc0, scoring='accuracy', param_grid=default_params, return_train_score=True, verbose=1, cv=3)
clf0.fit(X, y_train.values.ravel())

#results dataframe
df = pd.DataFrame(clf0.cv_results_)

#predictions - inputs to confusion matrix
train_predictions = clf0.predict(X)
test_predictions = clf0.predict(X_v)

#confusion matrices
cfm_train = confusion_matrix(y_train, train_predictions)
cfm_test = confusion_matrix(y_val, test_predictions)

#accuracy scores
accs_train = accuracy_score(y_train, train_predictions)
accs_test = accuracy_score(y_val, test_predictions)

#F1 scores for each train/test label
f1s_train_p1 = f1_score(y_train, train_predictions, pos_label=1)
f1s_train_p0 = f1_score(y_train, train_predictions, pos_label=0)
f1s_test_p1 = f1_score(y_val, test_predictions, pos_label=1)
f1s_test_p0 = f1_score(y_val, test_predictions, pos_label=0)


#Area Under the Receiver Operating Characteristic Curve
test_ras = roc_auc_score(y_val, clf0.predict_proba(X_v)[:,1])
#best parameters
bp = clf0.best_params_
#storing computed values in results dictionary
results_dict['xgbc0'] = {'iterable_parameter': np.nan,
                         'cv_results': df.copy(),
                         'cfm_train': cfm_train,
                         'cfm_test': cfm_test,
                         'train_accuracy': accs_train,
                         'test_accuracy': accs_test,
                         'train F1-score label 1': f1s_train_p1,
                         'train F1-score label 0': f1s_train_p0,
                         'test F1-score label 1': f1s_test_p1,
                         'test F1-score label 0': f1s_test_p0,
                         'test roc auc score': test_ras,
                         'best_params': bp}

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [30]:
test_data = pd.read_csv('test.csv')

In [31]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [32]:
d = data_preproccessing(test_data)

In [33]:
test = d.drop("PassengerId",axis = 1)

results = clf0.predict(test)

In [34]:
def make_sub(result:np.array, name_of_df:str):
    res = {'PassengerId': d["PassengerId"], 'Transported': result}
    results = pd.DataFrame(data=res)
    results["Transported"] = results["Transported"].astype(bool)
    results.to_csv(name_of_df, index=False)

res = {'PassengerId': d["PassengerId"], 'Transported': results}

results = pd.DataFrame(data=res)

results["Transported"] = results["Transported"].astype(bool)

results

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [85]:
results.to_csv("submission_xgboost.csv", index=False)

#  Training RandomForest

In [103]:
rfc = RandomForestClassifier(max_depth=12, random_state=42)

In [104]:
rfc.fit(X, y_train)

RandomForestClassifier(max_depth=12, random_state=42)

In [105]:
y_pred = rfc.predict(X_v)

In [106]:
print('RandomForestClassifier Testing Accurancy: ', accuracy_score(y_val, y_pred))

RandomForestClassifier Testing Accurancy:  0.7883435582822086


In [107]:
results_from_rfc = rfc.predict(test)

In [113]:
make_sub(results_from_rfc, 'submission_from_rfc.csv')

In [80]:
cbc = CatBoostClassifier(
    iterations=100, 
    learning_rate=0.3,
    loss_function='CrossEntropy',
    eval_metric = 'AUC',
    depth = 6
)

cbc.fit(X, y_train,
        verbose=False
)

y_pred_cbc = cbc.predict(X_v)

print('CatBoostClassifier Testing Accurancy: ', accuracy_score(y_val, y_pred_cbc))

In [192]:
make_sub(results_from_rfc, 'submission_from_catboost.csv')

## 2 type

In [126]:
"""

"""
train_data = pd.read_csv("train.csv")

In [127]:
for i in train_data.columns:
    if train_data[i].dtypes == 'object':
        train_data[i].fillna(train_data[i].mode()[0], inplace=True)
    else:
        train_data[i].fillna(train_data[i].median(), inplace=True)
        
        
def ben(x):
    if x == True:
        return(1)
    else:
        return(0)
train_data['VIP'] = train_data['VIP'].map(lambda x: ben(x))
train_data['CryoSleep'] = train_data['CryoSleep'].map(lambda x: ben(x))

train_data["Transported"] = train_data["Transported"].map(lambda x: ben(x))

y_train = train_data["Transported"]

train_data.drop("Transported",axis = 1,inplace = True)

train_data.drop("Name", axis = 1, inplace = True)

train_data.head()

HomePlanet = pd.get_dummies(train_data.HomePlanet, prefix='HomePlanet')
train_data = pd.concat([train_data, HomePlanet], axis=1)

train_data.drop(['HomePlanet'],axis=1,inplace = True)
train_data = one_hot(train_data,'Destination')
b = []

for i in train_data['Cabin']:
    if i[-1:] == "P":
        b.append("P")
    else:
        b.append("S")

train_data["CabinP/S"] = b

c = []
for i in train_data['Cabin']:
    c.append(i[:1])
train_data["CabinA_T"] = c

train_data = one_hot(train_data,"CabinP/S")
train_data = one_hot(train_data,"CabinA_T")
train_data.drop(["Cabin"], axis = 1, inplace = True)

In [128]:
train_data

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,...,CabinP/S_P,CabinP/S_S,CabinA_T_A,CabinA_T_B,CabinA_T_C,CabinA_T_D,CabinA_T_E,CabinA_T_F,CabinA_T_G,CabinA_T_T
0,0001_01,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,...,1,0,0,1,0,0,0,0,0,0
1,0002_01,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,...,0,1,0,0,0,0,0,1,0,0
2,0003_01,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,...,0,1,1,0,0,0,0,0,0,0
3,0003_02,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,...,0,1,1,0,0,0,0,0,0,0
4,0004_01,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0,...,1,0,1,0,0,0,0,0,0,0
8689,9278_01,1,18.0,0,0.0,0.0,0.0,0.0,0.0,1,...,0,1,0,0,0,0,0,0,1,0
8690,9279_01,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,1,...,0,1,0,0,0,0,0,0,1,0
8691,9280_01,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0,...,0,1,0,0,0,0,1,0,0,0


In [129]:
train_data.drop("PassengerId",axis = 1,inplace = True)

In [130]:
state = 12  
test_size = 0.30  


X_train, X_val, y_train, y_val = train_test_split(train_data, y_train, test_size=test_size, random_state=state)

In [131]:
X_train

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,...,CabinP/S_P,CabinP/S_S,CabinA_T_A,CabinA_T_B,CabinA_T_C,CabinA_T_D,CabinA_T_E,CabinA_T_F,CabinA_T_G,CabinA_T_T
4667,1,2.0,0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,1,0,0,0,0,0,0,1,0
3109,0,22.0,0,0.0,5.0,0.0,12.0,6111.0,0,1,...,0,1,0,0,1,0,0,0,0,0
939,0,3.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,1,0,0
7533,0,27.0,0,0.0,0.0,846.0,59.0,0.0,1,0,...,0,1,0,0,0,0,1,0,0,0
4102,0,36.0,0,735.0,0.0,2.0,0.0,0.0,1,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0,18.0,0,0.0,0.0,1359.0,0.0,0.0,1,0,...,0,1,0,0,0,0,0,1,0,0
3714,0,4.0,0,0.0,0.0,0.0,0.0,0.0,1,0,...,1,0,0,0,0,0,0,0,1,0
7409,1,24.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,1,0,0
3325,1,19.0,0,0.0,0.0,0.0,0.0,0.0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [133]:
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [134]:
print(rfc.score(X_val, y_val))
print(gbc.score(X_val, y_val))

0.7051380368098159
0.7921779141104295


C:\Users\Anykey\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [135]:
test_data = pd.read_csv("test.csv")

In [136]:
for i in test_data.columns:
    if test_data[i].dtypes == 'object':
        test_data[i].fillna(test_data[i].mode()[0], inplace=True)
    else:
        test_data[i].fillna(test_data[i].median(), inplace=True)
        
        
def ben(x):
    if x == True:
        return(1)
    else:
        return(0)
test_data['VIP'] = test_data['VIP'].map(lambda x: ben(x))
test_data['CryoSleep'] = test_data['CryoSleep'].map(lambda x: ben(x))

#train_data["Transported"] = train_data["Transported"].map(lambda x: ben(x))

#y_train = train_data["Transported"]

#train_data.drop("Transported",axis = 1,inplace = True)

test_data.drop("Name", axis = 1, inplace = True)


HomePlanet = pd.get_dummies(test_data.HomePlanet, prefix='HomePlanet')
test_data = pd.concat([test_data, HomePlanet], axis=1)

test_data.drop(['HomePlanet'],axis=1,inplace = True)
test_data = one_hot(test_data,'Destination')
b = []

for i in test_data['Cabin']:
    if i[-1:] == "P":
        b.append("P")
    else:
        b.append("S")

test_data["CabinP/S"] = b

c = []
for i in test_data['Cabin']:
    c.append(i[:1])
test_data["CabinA_T"] = c

test_data = one_hot(test_data,"CabinP/S")
test_data = one_hot(test_data,"CabinA_T")
test_data.drop(["Cabin"], axis = 1, inplace = True)

In [138]:
id_test = test_data["PassengerId"]

In [139]:
test_data.drop("PassengerId",axis = 1,inplace = True)

In [141]:
res = gbc.predict(test_data)

In [150]:
d

0       0013_01
1       0018_01
2       0019_01
3       0021_01
4       0023_01
         ...   
4272    9266_02
4273    9269_01
4274    9271_01
4275    9273_01
4276    9277_01
Name: PassengerId, Length: 4277, dtype: object

In [151]:

res = {'PassengerId': d, 'Transported': res}

results = pd.DataFrame(data=res)

results["Transported"] = results["Transported"].astype(bool)

results

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [154]:
results.to_csv('results_10_07.csv', index=False)